In [44]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [45]:
_INPUT_DATA_PATH = ['./FC_2021-03.csv',
                    './FC_2021-04.csv',
                    './FC_2021-05.csv',
                    './FC_2021-06.csv']


In [46]:
df = pd.DataFrame()
for data_path in _INPUT_DATA_PATH:
    temp_df = pd.read_csv(data_path, low_memory=False, dtype={
                     'BKG_DATE': str, 'BKG_TIME': str, 'POST_ZONE': str
                 })
    df = pd.concat([df, temp_df], ignore_index=True)

    print(data_path, "\t", temp_df.shape)

print(df.info())

./FC_2021-03.csv 	 (309352, 32)
./FC_2021-04.csv 	 (349588, 32)
./FC_2021-05.csv 	 (450522, 32)
./FC_2021-06.csv 	 (676912, 32)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1786374 entries, 0 to 1786373
Data columns (total 32 columns):
 #   Column              Dtype  
---  ------              -----  
 0   CORP_ID             object 
 1   REF_ORD_NO          object 
 2   BKG_NO              object 
 3   BKG_TYP             int64  
 4   BKG_DATE            object 
 5   BKG_TIME            object 
 6   SHPR_CD             int64  
 7   INV_AMT             float64
 8   ITEM_SEQ            int64  
 9   ITEM_CD             object 
 10  BRAND_NM            object 
 11  ITEM_QTY            int64  
 12  ITEM_AMT            float64
 13  IF_YN               object 
 14  ORDER_CRT_DATETIME  float64
 15  DLV_DV              float64
 16  REF_ITEM_SEQ        object 
 17  ORDER_IDX           object 
 18  ORDER_YN            object 
 19  DLVPREARRBRANCD     object 
 20  DLVPREARREMPNICKNM  object 


In [47]:
_cols = {'CORP_ID', 'BKG_NO', 'INS_ID', 'INS_DATE', 'INS_TIME', 'BRAND_NM',
         'ORDER_IDX', 'ORDER_YN', 'SPLIT_EXEC_YN', 'DLVCLSFCD', 'DLVSUBCLSFCD', 
         'DLVPREARRBRANCD', 'DLVPREARREMPNICKNM', 'DLV_DV', 'REF_ITEM_SEQ', 'IF_YN',
         'ORDER_CRT_DATETIME', 'SHPR_ADDR_1', 'SHPR_ADDR_2', 'CNEE_ADDR_1', 'CNEE_ADDR_2','BKG_TYP','ITEM_SEQ','REF_ORD_NO','POST_ZONE','ITEM_QTY','BKG_TIME'}

fc = df.loc[df['CORP_ID'] == 'KX007'].drop(columns=_cols)
fc.head(5)

,BKG_DATE,SHPR_CD,INV_AMT,ITEM_CD,ITEM_AMT
0,20210303,90001302,22400.0,134265392,14900.0
1,20210303,90001441,20900.0,4610261043_2,20900.0
2,20210303,90001441,37900.0,4936599538_1,37900.0
3,20210303,90001542,69800.0,3858391637_1,69800.0
4,20210303,90001341,73700.0,4631307460,39800.0


In [48]:
fc['BKG_DATE'] = pd.to_datetime(fc['BKG_DATE'])
#fc['BKG_TIME'] = pd.to_datetime(fc['BKG_TIME'], format="%H%M%S").dt.time

In [49]:
# 1441 고객사

fc_1302 = fc[fc['SHPR_CD'] == 90001582]
fc_1302.drop(columns='SHPR_CD')

,BKG_DATE,INV_AMT,ITEM_CD,ITEM_AMT
309160,2021-03-31,39600.0,4610252177_3,NaN
309161,2021-03-31,39600.0,4624382111_3,NaN
309165,2021-03-31,20100.0,161523952_2,20100.0
309166,2021-03-31,20100.0,161523952_1,20100.0
309167,2021-03-31,87200.0,304489410_4,87200.0
...,...,...,...,...
1786217,2021-06-30,50800.0,677756200_2,50800.0
1786249,2021-06-30,22500.0,5497354651_4,22500.0
1786298,2021-06-30,24600.0,5497354651_3,24600.0
1786301,2021-06-29,19500.0,161523952_6,19500.0


In [50]:
# 1
fc_1302_inv = fc_1302[['BKG_DATE','INV_AMT']]
fc_1302_inv = fc_1302_inv.groupby(['BKG_DATE']).sum()
fc_1302_inv

,INV_AMT
BKG_DATE,
2021-03-31,8735600.0
2021-04-01,12341700.0
2021-04-02,11160000.0
2021-04-03,12628600.0
2021-04-04,14079800.0
...,...
2021-06-26,35450750.0
2021-06-27,88032150.0
2021-06-28,168012700.0


In [51]:
fc_1302_week = fc_1302_inv.resample(rule='1W')
fc_1302_week = fc_1302_week.sum()
fc_1302_week

,INV_AMT
BKG_DATE,
2021-04-04,58945700.0
2021-04-11,209661300.0
2021-04-18,200880300.0
2021-04-25,453104000.0
2021-05-02,193902300.0
2021-05-09,145775200.0
2021-05-16,219731850.0
2021-05-23,224581650.0
2021-05-30,363753700.0


In [52]:
fc_1302_inv = pd.concat([fc_1302_inv,fc_1302_week],axis=1)
fc_1302_inv = fc_1302_inv.fillna(method = 'bfill')
fc_1302_inv = fc_1302_inv[0:-1][:]
fc_1302_inv.columns = ['INV_AMT','INV_AMT_WEEK']                     

In [53]:
fc_1302_inv['INV_Q1'] = fc_1302_inv['INV_AMT_WEEK']*0.25
fc_1302_inv['INV_Q2'] = fc_1302_inv['INV_AMT_WEEK']*0.5
fc_1302_inv['INV_Q3'] = fc_1302_inv['INV_AMT_WEEK']*0.75
fc_1302_inv

,INV_AMT,INV_AMT_WEEK,INV_Q1,INV_Q2,INV_Q3
BKG_DATE,,,,,
2021-03-31,8735600.0,58945700.0,14736425.0,29472850.0,44209275.0
2021-04-01,12341700.0,58945700.0,14736425.0,29472850.0,44209275.0
2021-04-02,11160000.0,58945700.0,14736425.0,29472850.0,44209275.0
2021-04-03,12628600.0,58945700.0,14736425.0,29472850.0,44209275.0
2021-04-04,14079800.0,58945700.0,14736425.0,29472850.0,44209275.0
...,...,...,...,...,...
2021-06-26,35450750.0,389821900.0,97455475.0,194910950.0,292366425.0
2021-06-27,88032150.0,389821900.0,97455475.0,194910950.0,292366425.0
2021-06-28,168012700.0,290565000.0,72641250.0,145282500.0,217923750.0


In [54]:
fc_1302_inv = fc_1302_inv.drop(columns='INV_AMT_WEEK')
fc_1302_inv

,INV_AMT,INV_Q1,INV_Q2,INV_Q3
BKG_DATE,,,,
2021-03-31,8735600.0,14736425.0,29472850.0,44209275.0
2021-04-01,12341700.0,14736425.0,29472850.0,44209275.0
2021-04-02,11160000.0,14736425.0,29472850.0,44209275.0
2021-04-03,12628600.0,14736425.0,29472850.0,44209275.0
2021-04-04,14079800.0,14736425.0,29472850.0,44209275.0
...,...,...,...,...
2021-06-26,35450750.0,97455475.0,194910950.0,292366425.0
2021-06-27,88032150.0,97455475.0,194910950.0,292366425.0
2021-06-28,168012700.0,72641250.0,145282500.0,217923750.0


In [55]:
for i, row in fc_1302_inv.iterrows():
    INV_AMT = row['INV_AMT']
    q1 = row['INV_Q1']
    q2 = row['INV_Q2']
    q3 = row['INV_Q3']
    
    if INV_AMT < q1:
        row['INV_AMT'] = 1
    elif q1 <= INV_AMT < q2:
        row['INV_AMT'] = 2
    elif q2 <= INV_AMT < q3:
        row['INV_AMT'] = 3
    elif q3 <= INV_AMT:
        row['INV_AMT'] = 4
    else:
        row['INV_AMT'] = 0 # 오류확인
        
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

            INV_AMT      INV_Q1       INV_Q2       INV_Q3
BKG_DATE                                                 
2021-03-31      1.0  14736425.0   29472850.0   44209275.0
2021-04-01      1.0  14736425.0   29472850.0   44209275.0
2021-04-02      1.0  14736425.0   29472850.0   44209275.0
2021-04-03      1.0  14736425.0   29472850.0   44209275.0
2021-04-04      1.0  14736425.0   29472850.0   44209275.0
...             ...         ...          ...          ...
2021-06-26      1.0  97455475.0  194910950.0  292366425.0
2021-06-27      1.0  97455475.0  194910950.0  292366425.0
2021-06-28      3.0  72641250.0  145282500.0  217923750.0
2021-06-29      1.0  72641250.0  145282500.0  217923750.0
2021-06-30      1.0  72641250.0  145282500.0  217923750.0

[92 rows x 4 columns]


In [56]:
fc_1302_inv['INV_AMT'].value_counts()

1.0    83
2.0     8
3.0     1
Name: INV_AMT, dtype: int64

In [57]:
fc_1302_inv = fc_1302_inv.drop(columns=['INV_Q1','INV_Q2','INV_Q3'])
fc_1302_inv = fc_1302_inv.reset_index()
fc_1302_inv

,BKG_DATE,INV_AMT
0,2021-03-31,1.0
1,2021-04-01,1.0
2,2021-04-02,1.0
3,2021-04-03,1.0
4,2021-04-04,1.0
...,...,...
87,2021-06-26,1.0
88,2021-06-27,1.0
89,2021-06-28,3.0
90,2021-06-29,1.0


In [58]:
## item amt 

fc_1302_amt = fc_1302[['BKG_DATE','ITEM_AMT']]
fc_1302_amt = fc_1302_amt.groupby(['BKG_DATE']).sum()

fc_1302_week = fc_1302_amt.resample(rule='1W')
fc_1302_week = fc_1302_week.sum()


In [59]:
fc_1302_amt = pd.concat([fc_1302_amt,fc_1302_week],axis=1)
fc_1302_amt = fc_1302_amt.fillna(method = 'bfill')
fc_1302_amt = fc_1302_amt[0:-1][:]
fc_1302_amt.columns = ['ITEM_AMT','ITEM_AMT_WEEK'] 
fc_1302_amt

,ITEM_AMT,ITEM_AMT_WEEK
BKG_DATE,,
2021-03-31,7763500.0,54709650.0
2021-04-01,10831900.0,54709650.0
2021-04-02,13279700.0,54709650.0
2021-04-03,10741900.0,54709650.0
2021-04-04,12092650.0,54709650.0
...,...,...
2021-06-26,26507950.0,267367770.0
2021-06-27,59563300.0,267367770.0
2021-06-28,111948250.0,194497350.0


In [60]:
fc_1302_amt['ITEM_Q1'] = fc_1302_amt['ITEM_AMT_WEEK']*0.25
fc_1302_amt['ITEM_Q2'] = fc_1302_amt['ITEM_AMT_WEEK']*0.5
fc_1302_amt['ITEM_Q3'] = fc_1302_amt['ITEM_AMT_WEEK']*0.75
fc_1302_amt

,ITEM_AMT,ITEM_AMT_WEEK,ITEM_Q1,ITEM_Q2,ITEM_Q3
BKG_DATE,,,,,
2021-03-31,7763500.0,54709650.0,13677412.5,27354825.0,41032237.5
2021-04-01,10831900.0,54709650.0,13677412.5,27354825.0,41032237.5
2021-04-02,13279700.0,54709650.0,13677412.5,27354825.0,41032237.5
2021-04-03,10741900.0,54709650.0,13677412.5,27354825.0,41032237.5
2021-04-04,12092650.0,54709650.0,13677412.5,27354825.0,41032237.5
...,...,...,...,...,...
2021-06-26,26507950.0,267367770.0,66841942.5,133683885.0,200525827.5
2021-06-27,59563300.0,267367770.0,66841942.5,133683885.0,200525827.5
2021-06-28,111948250.0,194497350.0,48624337.5,97248675.0,145873012.5


In [61]:
fc_1302_amt = fc_1302_amt.drop(columns='ITEM_AMT_WEEK')
fc_1302_amt

,ITEM_AMT,ITEM_Q1,ITEM_Q2,ITEM_Q3
BKG_DATE,,,,
2021-03-31,7763500.0,13677412.5,27354825.0,41032237.5
2021-04-01,10831900.0,13677412.5,27354825.0,41032237.5
2021-04-02,13279700.0,13677412.5,27354825.0,41032237.5
2021-04-03,10741900.0,13677412.5,27354825.0,41032237.5
2021-04-04,12092650.0,13677412.5,27354825.0,41032237.5
...,...,...,...,...
2021-06-26,26507950.0,66841942.5,133683885.0,200525827.5
2021-06-27,59563300.0,66841942.5,133683885.0,200525827.5
2021-06-28,111948250.0,48624337.5,97248675.0,145873012.5


In [62]:
for i, row in fc_1302_amt.iterrows():
    ITEM_AMT = row['ITEM_AMT']
    q1 = row['ITEM_Q1']
    q2 = row['ITEM_Q2']
    q3 = row['ITEM_Q3']
    
    if ITEM_AMT < q1:
        row['ITEM_AMT'] = 1
    elif q1 <= ITEM_AMT < q2:
        row['ITEM_AMT'] = 2
    elif q2 <= ITEM_AMT < q3:
        row['ITEM_AMT'] = 3
    elif q3 <= ITEM_AMT:
        row['ITEM_AMT'] = 4
    else:
        row['ITEM_AMT'] = 0 # 오류확인
        
    fc_1302_amt.loc[i] = row
    
print(fc_1302_amt)

            ITEM_AMT     ITEM_Q1      ITEM_Q2      ITEM_Q3
BKG_DATE                                                  
2021-03-31       1.0  13677412.5   27354825.0   41032237.5
2021-04-01       1.0  13677412.5   27354825.0   41032237.5
2021-04-02       1.0  13677412.5   27354825.0   41032237.5
2021-04-03       1.0  13677412.5   27354825.0   41032237.5
2021-04-04       1.0  13677412.5   27354825.0   41032237.5
...              ...         ...          ...          ...
2021-06-26       1.0  66841942.5  133683885.0  200525827.5
2021-06-27       1.0  66841942.5  133683885.0  200525827.5
2021-06-28       3.0  48624337.5   97248675.0  145873012.5
2021-06-29       1.0  48624337.5   97248675.0  145873012.5
2021-06-30       1.0  48624337.5   97248675.0  145873012.5

[92 rows x 4 columns]


In [63]:
fc_1302_amt = fc_1302_amt.drop(columns=['ITEM_Q1','ITEM_Q2','ITEM_Q3'])
fc_1302_amt = fc_1302_amt.reset_index()
fc_1302_amt['ITEM_AMT'].value_counts()

1.0    84
2.0     7
3.0     1
Name: ITEM_AMT, dtype: int64

In [64]:
fc_1302_inv['ITEM_AMT'] = fc_1302_amt['ITEM_AMT']

In [65]:
fc_1302_inv['weekday'] = fc_1302_inv['BKG_DATE'].dt.dayofweek
fc_1302_inv['weekday'].unique()

array([2, 3, 4, 5, 6, 0, 1])

In [66]:
n = 0

for i in fc_1302_inv['weekday']:
    if fc_1302_inv['weekday'][n] == 0:
        fc_1302_inv['weekday'][n] = 1
    else:
        fc_1302_inv['weekday'][n] = 0
    n += 1
    if n == 122:
        break

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17562/1473546988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['weekday'][n] = 0
/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17562/1473546988.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['weekday'][n] = 1


In [67]:
from pytimekr import pytimekr
holiday = pytimekr.holidays(year=2021)

fc_1302_inv['holiday'] = fc_1302_inv['BKG_DATE']

In [68]:
# 휴일 변수 할당

a='2021-03-01'
a= pd.to_datetime(a)
b='2021-05-05'
b= pd.to_datetime(b)
c='2021-05-19'
c= pd.to_datetime(c)
d='2021-06-06'
d= pd.to_datetime(d)

In [69]:
# 휴일 

for i, row in fc_1302_inv.iterrows():
    holiday = row['holiday']

    if holiday == a :
         row['holiday'] = 1
    elif holiday == b :
        row['holiday'] = 1
    elif holiday == c :
        row['holiday'] = 1
    elif holiday == d :
        row['holiday'] = 1        
    else:
         row['holiday'] = 0
   
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

     BKG_DATE  INV_AMT  ITEM_AMT  weekday holiday
0  2021-03-31      1.0       1.0        0       0
1  2021-04-01      1.0       1.0        0       0
2  2021-04-02      1.0       1.0        0       0
3  2021-04-03      1.0       1.0        0       0
4  2021-04-04      1.0       1.0        0       0
..        ...      ...       ...      ...     ...
87 2021-06-26      1.0       1.0        0       0
88 2021-06-27      1.0       1.0        0       0
89 2021-06-28      3.0       3.0        1       0
90 2021-06-29      1.0       1.0        0       0
91 2021-06-30      1.0       1.0        0       0

[92 rows x 5 columns]


In [70]:
# 휴일 하루 전

fc_1302_inv['holiday_diff'] = fc_1302_inv['BKG_DATE']

In [71]:
# 휴일 전 변수 할당

q='2021-02-28'
q= pd.to_datetime(q)
w='2021-05-04'
w= pd.to_datetime(w)
e='2021-05-18'
e= pd.to_datetime(e)
r='2021-06-05'
r= pd.to_datetime(r)

In [72]:
# 휴일 하루 전

fc_1302_inv['holiday_diff'] = fc_1302_inv['BKG_DATE']


# 휴일 전 변수 할당

q='2021-02-28'
q= pd.to_datetime(q)
w='2021-05-04'
w= pd.to_datetime(w)
e='2021-05-18'
e= pd.to_datetime(e)
r='2021-06-05'
r= pd.to_datetime(r)



for i, row in fc_1302_inv.iterrows():
    holiday_diff = row['holiday_diff']

    if holiday_diff == q :
         row['holiday_diff'] = 1
    elif holiday_diff == w :
        row['holiday_diff'] = 1
    elif holiday_diff == e :
        row['holiday_diff'] = 1
    elif holiday_diff == r :
        row['holiday_diff'] = 1        
    else:
         row['holiday_diff'] = 0
   
    fc_1302_inv.loc[i] = row
    
print(fc_1302_inv)

     BKG_DATE  INV_AMT  ITEM_AMT  weekday holiday holiday_diff
0  2021-03-31      1.0       1.0        0       0            0
1  2021-04-01      1.0       1.0        0       0            0
2  2021-04-02      1.0       1.0        0       0            0
3  2021-04-03      1.0       1.0        0       0            0
4  2021-04-04      1.0       1.0        0       0            0
..        ...      ...       ...      ...     ...          ...
87 2021-06-26      1.0       1.0        0       0            0
88 2021-06-27      1.0       1.0        0       0            0
89 2021-06-28      3.0       3.0        1       0            0
90 2021-06-29      1.0       1.0        0       0            0
91 2021-06-30      1.0       1.0        0       0            0

[92 rows x 6 columns]


In [73]:
### ITEM_CD count

cd = fc_1302[['BKG_DATE','ITEM_CD']]
cd = cd.groupby(['BKG_DATE']).count()
cd = cd.reset_index()
fc_1302_inv['ITEM_CD_COUNT'] = cd['ITEM_CD']

In [74]:
diff1 = fc_1302_inv['INV_AMT'][:-1]
fc_1302_inv['INV_AMT_DIFF'] = fc_1302_inv['INV_AMT']
fc_1302_inv['INV_AMT_DIFF'].iloc[1:] = diff1
fc_1302_inv

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17562/3734826041.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['INV_AMT_DIFF'].iloc[1:] = diff1


,BKG_DATE,INV_AMT,ITEM_AMT,weekday,holiday,holiday_diff,ITEM_CD_COUNT,INV_AMT_DIFF
0,2021-03-31,1.0,1.0,0,0,0,186,1.0
1,2021-04-01,1.0,1.0,0,0,0,245,1.0
2,2021-04-02,1.0,1.0,0,0,0,304,1.0
3,2021-04-03,1.0,1.0,0,0,0,241,1.0
4,2021-04-04,1.0,1.0,0,0,0,290,1.0
...,...,...,...,...,...,...,...,...
87,2021-06-26,1.0,1.0,0,0,0,821,1.0
88,2021-06-27,1.0,1.0,0,0,0,1943,1.0
89,2021-06-28,3.0,3.0,1,0,0,3201,1.0
90,2021-06-29,1.0,1.0,0,0,0,1350,3.0


In [75]:
diff2 = fc_1302_inv['ITEM_AMT'][:-1]
fc_1302_inv['ITEM_AMT_DIFF'] = fc_1302_inv['ITEM_AMT']
fc_1302_inv['ITEM_AMT_DIFF'].iloc[1:] = diff2
fc_1302_inv

/var/folders/6y/zgxv63kx28z7cl3_zq45knx00000gn/T/ipykernel_17562/3619347445.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fc_1302_inv['ITEM_AMT_DIFF'].iloc[1:] = diff2


,BKG_DATE,INV_AMT,ITEM_AMT,weekday,holiday,holiday_diff,ITEM_CD_COUNT,INV_AMT_DIFF,ITEM_AMT_DIFF
0,2021-03-31,1.0,1.0,0,0,0,186,1.0,1.0
1,2021-04-01,1.0,1.0,0,0,0,245,1.0,1.0
2,2021-04-02,1.0,1.0,0,0,0,304,1.0,1.0
3,2021-04-03,1.0,1.0,0,0,0,241,1.0,1.0
4,2021-04-04,1.0,1.0,0,0,0,290,1.0,1.0
...,...,...,...,...,...,...,...,...,...
87,2021-06-26,1.0,1.0,0,0,0,821,1.0,1.0
88,2021-06-27,1.0,1.0,0,0,0,1943,1.0,1.0
89,2021-06-28,3.0,3.0,1,0,0,3201,1.0,1.0
90,2021-06-29,1.0,1.0,0,0,0,1350,3.0,3.0


In [76]:
fc_1302_inv = fc_1302_inv[['BKG_DATE','INV_AMT','INV_AMT_DIFF','ITEM_AMT','ITEM_AMT_DIFF','weekday','holiday','holiday_diff','ITEM_CD_COUNT']]
fc_1302_inv

,BKG_DATE,INV_AMT,INV_AMT_DIFF,ITEM_AMT,ITEM_AMT_DIFF,weekday,holiday,holiday_diff,ITEM_CD_COUNT
0,2021-03-31,1.0,1.0,1.0,1.0,0,0,0,186
1,2021-04-01,1.0,1.0,1.0,1.0,0,0,0,245
2,2021-04-02,1.0,1.0,1.0,1.0,0,0,0,304
3,2021-04-03,1.0,1.0,1.0,1.0,0,0,0,241
4,2021-04-04,1.0,1.0,1.0,1.0,0,0,0,290
...,...,...,...,...,...,...,...,...,...
87,2021-06-26,1.0,1.0,1.0,1.0,0,0,0,821
88,2021-06-27,1.0,1.0,1.0,1.0,0,0,0,1943
89,2021-06-28,3.0,1.0,3.0,1.0,1,0,0,3201
90,2021-06-29,1.0,3.0,1.0,3.0,0,0,0,1350


In [77]:
fc_1302_inv[70:]

,BKG_DATE,INV_AMT,INV_AMT_DIFF,ITEM_AMT,ITEM_AMT_DIFF,weekday,holiday,holiday_diff,ITEM_CD_COUNT
70,2021-06-09,1.0,1.0,1.0,1.0,0,0,0,967
71,2021-06-10,1.0,1.0,1.0,1.0,0,0,0,1030
72,2021-06-11,1.0,1.0,1.0,1.0,0,0,0,717
73,2021-06-12,1.0,1.0,1.0,1.0,0,0,0,739
74,2021-06-13,1.0,1.0,1.0,1.0,0,0,0,1296
75,2021-06-14,1.0,1.0,1.0,1.0,1,0,0,1358
76,2021-06-15,1.0,1.0,1.0,1.0,0,0,0,1103
77,2021-06-16,1.0,1.0,1.0,1.0,0,0,0,1102
78,2021-06-17,1.0,1.0,1.0,1.0,0,0,0,861
79,2021-06-18,1.0,1.0,1.0,1.0,0,0,0,742


In [78]:
fc_1302_train = fc_1302_inv.iloc[:62]
fc_1302_test = fc_1302_inv.iloc[62:]

In [79]:
fc_1302_train.to_csv('fc_1582_train.csv',index=False)
fc_1302_test.to_csv('fc_1582_test.csv',index=False)